# Tensorflow Model Optimization Toolkit (TMO)

Dalam buku catatan ini, kami akan menunjukkan cara menggunakan TMO untuk mengoptimalkan model untuk penerapan. Kami melatih model pada set data MNIST lalu mengoptimalkannya menggunakan TMO. Kami kemudian akan membandingkan ukuran dan akurasi model yang dioptimalkan dengan model asli.

## Siapkan TMO

Pertama, kita instal TMO dan impor paket yang dibutuhkan.

In [1]:
%pip install -q tensorflow
%pip install -q tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 597.2 kB/s eta 0:00:00


In [2]:
import tensorflow as tf
import tensorflow_model_optimization as tfmot
from tensorflow import keras
import pathlib
import numpy as np


## Kuantisasi Setelah Pelatihan

Alat kuantisasi setelah pelatihan mengonversi bobot model yang telah dilatih dari presisi 32 bit ke presisi 8 bit. Alat ini mengonversi model TensorFlow float yang sudah dilatih saat kita mengonversinya ke format TensorFlow Lite menggunakan [TensorFlow Lite Converter](https://www.tensorflow.org/lite/models/convert/).

### Memuat dataset MNIST

Kami memuat dataset MNIST dari Keras dan mempersiapkannya untuk pelatihan.

In [3]:
# Memuat dataset MNIST
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Menormalisasi gambar input sehingga setiap nilai piksel berada di antara 0 dan 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

11490434/11490434 [==============================] - 0s 0us/step


### Melatih Model

Selanjutnya, kami mendefinisikan model CNN dan melatihnya pada dataset MNIST.

In [4]:
# Mendefinisikan arsitektur model
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),  # Lapisan input dengan bentuk (28, 28)
  keras.layers.Reshape(target_shape=(28, 28, 1)),  # Mengubah bentuk input menjadi (28, 28, 1)
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation=tf.nn.relu),  # Lapisan konvolusi dengan 12 filter dan ukuran kernel 3x3
  keras.layers.MaxPooling2D(pool_size=(2, 2)),  # Lapisan pooling maksimum dengan ukuran 2x2
  keras.layers.Flatten(),  # Mengubah array multidimensi menjadi satu dimensi
  keras.layers.Dense(10)  # Lapisan dense dengan 10 neuron (sesuai dengan jumlah kelas digit)
])

# Melatih model klasifikasi digit
model.compile(optimizer='adam',  # Optimizer Adam
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Fungsi kerugian untuk klasifikasi
              metrics=['accuracy'])  # Metrik akurasi
model.fit(
  train_images,  # Data pelatihan
  train_labels,  # Label pelatihan
  epochs=1,  # Jumlah epoch pelatihan
  validation_data=(test_images, test_labels)  # Data validasi
)

1875/1875 [==============================] - 15s 6ms/step - loss: 0.2764 - accuracy: 0.9231 - val_loss: 0.1339 - val_accuracy: 0.9623


### Mengonversi Model ke TFLite

Setelah melatih model, kita mengonversinya ke format [TFLite](https://www.tensorflow.org/lite/guide) dan kemudian melakukan kuantisasi selama proses konversi.

In [5]:
tflite_models_dir = pathlib.Path("notebooks/Unit 9 - Model Optimization/models")
tflite_models_dir.mkdir(exist_ok=True, parents=True)  # Membuat direktori untuk menyimpan model jika belum ada
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Menginisialisasi konverter dari model Keras

# Tanpa kuantisasi
tflite_model = converter.convert()  # Mengonversi model ke format TFLite
tflite_model_file = tflite_models_dir/"original_model.tflite"  # Menentukan nama file untuk model TFLite asli
tflite_model_file.write_bytes(tflite_model)  # Menyimpan model ke file

# Dengan kuantisasi
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Mengaktifkan optimisasi default untuk kuantisasi
tflite_quant_model = converter.convert()  # Mengonversi model ke format TFLite dengan kuantisasi
tflite_model_quant_file = tflite_models_dir/"quantized_model.tflite"  # Menentukan nama file untuk model TFLite dengan kuantisasi
tflite_model_quant_file.write_bytes(tflite_quant_model)  # Menyimpan model yang telah dikuantisasi ke file


23968

### Periksa Ukuran Model

Ukuran model terkuantisasi jauh lebih kecil daripada model asli.

In [6]:
%ls -lh {tflite_models_dir}

ls: cannot access 'notebooks/Unit': No such file or directory
ls: cannot access '9': No such file or directory
ls: cannot access '-': No such file or directory
ls: cannot access 'Model': No such file or directory
ls: cannot access 'Optimization/models': No such file or directory


### Periksa Akurasi Model

Selanjutnya, kami mengevaluasi akurasi model terkuantisasi pada kumpulan data pengujian dan membandingkannya dengan model asli.
Berdasarkan hasil, kita dapat melihat bahwa akurasi model terkuantisasi sangat mendekati model asli.

In [7]:
# Fungsi pembantu untuk mengevaluasi model TF Lite menggunakan dataset "test".
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]  # Mendapatkan indeks input model
    output_index = interpreter.get_output_details()[0]["index"]  # Mendapatkan indeks output model

    # Melakukan prediksi pada setiap gambar dalam dataset "test".
    prediction_digits = []
    for test_image in test_images:
        # Pra-pemrosesan: tambahkan dimensi batch dan ubah tipe data ke float32
        # agar sesuai dengan format data input model.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)  # Menetapkan tensor input

        # Menjalankan inferensi.
        interpreter.invoke()

        # Pasca-pemrosesan: hapus dimensi batch dan temukan digit dengan
        # probabilitas tertinggi.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])  # Mendapatkan indeks dengan nilai maksimum
        prediction_digits.append(digit)

    # Membandingkan hasil prediksi dengan label ground truth untuk menghitung akurasi.
    accurate_count = 0
    for index in range(len(prediction_digits)):
        if prediction_digits[index] == test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)  # Menghitung akurasi

    return accuracy


# Menginisialisasi interpreter untuk model asli.
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()  # Mengalokasikan tensor
print("Akurasi model asli = ", evaluate_model(interpreter))  # Evaluasi model asli

# Menginisialisasi interpreter untuk model kuantisasi.
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()  # Mengalokasikan tensor
print("Akurasi model kuantisasi = ", evaluate_model(interpreter_quant))  # Evaluasi model kuantisasi


Akurasi model asli =  0.9623
Akurasi model kuantisasi =  0.9626


## Pemangkasan

Pemangkasan adalah teknik untuk mengurangi ukuran model dengan membuang bobot yang tidak penting. Hal ini ditentukan oleh besarnya bobot. Kita dapat menggunakan pemangkasan saat melatih model untuk mengurangi ukuran model.

In [8]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude  # Mengimpor fungsi pruning

# Menghitung langkah akhir untuk menyelesaikan pruning setelah 2 epoch.
batch_size = 128
epochs = 2
validation_split = 0.1  # 10% dari dataset pelatihan akan digunakan sebagai dataset validasi.

num_images = train_images.shape[0] * (1 - validation_split)  # Jumlah gambar untuk pelatihan
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs  # Langkah akhir pruning

# Mendefinisikan model untuk pruning.
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.50,  # Tingkat sparsitas awal
        final_sparsity=0.80,  # Tingkat sparsitas akhir
        begin_step=0,  # Langkah awal untuk memulai pruning
        end_step=end_step  # Langkah akhir untuk menyelesaikan pruning
    )
}

# Membuat model yang mendukung pruning.
model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` membutuhkan recompilasi model.
model_for_pruning.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Menampilkan ringkasan model.
print(model_for_pruning.summary())

# Mendefinisikan callback untuk memperbarui langkah pruning.
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
]

# Melatih model yang mendukung pruning.
model_for_pruning.fit(
    train_images, train_labels,
    batch_size=batch_size, epochs=epochs, validation_split=validation_split,
    callbacks=callbacks
)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n (PruneLowMagnitude)                                           
                                                        

### Bandingkan Akurasi

Kita dapat melihat bahwa akurasi model yang dipangkas sangat mendekati model asli.

In [9]:
# Mengevaluasi akurasi model awal (tanpa pruning).
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

# Mengevaluasi akurasi model setelah pruning.
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
    test_images, test_labels, verbose=0)

# Menampilkan hasil akurasi.
print('Akurasi model awal:', baseline_model_accuracy)
print('Akurasi model setelah pruning:', model_for_pruning_accuracy)


Akurasi model awal: 0.9646999835968018
Akurasi model setelah pruning: 0.9646999835968018


### Bandingkan Ukuran Model

Terakhir, kami membandingkan ukuran model yang dipangkas dengan model asli.

In [10]:
# Menghapus pruning dari model untuk menyiapkan model yang telah dipangkas.
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

# Mengonversi model yang telah dipangkas ke format TFLite.
pruning_converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = pruning_converter.convert()

# Menyimpan model TFLite yang telah dipangkas ke dalam file.
pruned_model_file = tflite_models_dir/"pruned_model.tflite"
pruned_model_file.write_bytes(pruned_tflite_model)


84616

In [11]:
%ls -lh {tflite_models_dir}

ls: cannot access 'notebooks/Unit': No such file or directory
ls: cannot access '9': No such file or directory
ls: cannot access '-': No such file or directory
ls: cannot access 'Model': No such file or directory
ls: cannot access 'Optimization/models': No such file or directory
